In [1]:
from datetime import datetime, timezone

from confflow import (
    AnyOf,
    BooleanField,
    Booleanlist,
    BytesField,
    Byteslist,
    DateField,
    Datelist,
    FloatField,
    Floatlist,
    IntegerField,
    Integerlist,
    Manager,
    OneOf,
    Schema,
    StringField,
    Stringlist,
)


In [2]:
schemas: list[Schema] = [
    Schema("service_mesh", description="Service mesh and networking configuration")
    .add(
        StringField(
            "provider",
            description="Service mesh provider",
            default="istio",
            enum=["istio", "linkerd", "consul", "envoy"],
        ),
    )
    .add(
        Schema("mtls", description="Mutual TLS configuration")
        .add(
            BooleanField(
                "enabled",
                description="Enable mTLS",
                default=True,
            ),
        )
        .add(
            Byteslist(
                "trusted_ca_certs",
                description="List of trusted CA certificates (PEM format)",
                default=[
                    b"-----BEGIN CERTIFICATE-----\nMIIC...",
                    b"-----BEGIN CERTIFICATE-----\nMIID...",
                ],
                min_length=1,
                max_length=10,
            ),
        )
        .add(
            BytesField(
                "service_cert",
                description="Service certificate",
                default=b"-----BEGIN CERTIFICATE-----\nMIIE...",
            ),
        )
        .add(
            IntegerField(
                "cert_rotation_days",
                description="Certificate rotation period in days",
                default=30,
                ge=1,
                le=365,
            ),
        ),
    )
    .add(
        Schema("traffic_policy", description="Traffic management policies")
        .add(
            FloatField(
                "timeout_seconds",
                description="Default request timeout",
                default=30.0,
                gt=0.0,
                le=300.0,
            ),
        )
        .add(
            IntegerField(
                "max_retries",
                description="Maximum retry attempts",
                default=3,
                ge=0,
                le=10,
            ),
        )
        .add(
            Floatlist(
                "circuit_breaker_thresholds",
                description="Error rate thresholds for circuit breaking (percentages)",
                default=[50.0, 75.0, 90.0],
                min_length=1,
                max_length=5,
                item_ge=0.0,
                item_le=100.0,
            ),
        ),
    ),
    Schema("deployment", description="Deployment strategy configuration")
    .add(
        DateField(
            "next_deployment_window",
            description="Next scheduled deployment window",
            default=datetime(2025, 10, 15, 2, 0, 0, tzinfo=timezone.utc),
        ),
    )
    .add(
        Datelist(
            "maintenance_windows",
            description="Scheduled maintenance windows",
            default=[
                datetime(2025, 10, 15, 2, 0, 0, tzinfo=timezone.utc),
                datetime(2025, 11, 15, 2, 0, 0, tzinfo=timezone.utc),
                datetime(2025, 12, 15, 2, 0, 0, tzinfo=timezone.utc),
            ],
            min_length=1,
            max_length=12,
        ),
    )
    .add(
        OneOf(
            Schema("blue_green", description="Blue-Green deployment strategy")
            .add(
                IntegerField(
                    "verification_time_minutes",
                    description="Time to verify new version before cutover",
                    default=15,
                    ge=1,
                    le=120,
                ),
            )
            .add(
                BooleanField(
                    "auto_rollback",
                    description="Automatically rollback on errors",
                    default=True,
                ),
            )
            .add(
                FloatField(
                    "error_threshold_percent",
                    description="Error rate threshold for rollback",
                    default=5.0,
                    ge=0.0,
                    le=100.0,
                ),
            ),
            Schema("canary", description="Canary deployment strategy")
            .add(
                Integerlist(
                    "traffic_increments",
                    description="Traffic percentages for each canary stage",
                    default=[10, 25, 50, 100],
                    min_length=2,
                    max_length=10,
                    item_gt=0,
                    item_le=100,
                ),
            )
            .add(
                IntegerField(
                    "stage_duration_minutes",
                    description="Duration of each canary stage",
                    default=30,
                    ge=5,
                    le=240,
                ),
            )
            .add(
                Stringlist(
                    "metrics_to_monitor",
                    description="Metrics to monitor during canary",
                    default=["error_rate", "latency_p99", "cpu_usage"],
                    min_length=1,
                    max_length=20,
                ),
            ),
            Schema("rolling", description="Rolling update strategy")
            .add(
                IntegerField(
                    "max_surge",
                    description="Maximum pods above desired count",
                    default=1,
                    ge=0,
                    le=10,
                ),
            )
            .add(
                IntegerField(
                    "max_unavailable",
                    description="Maximum unavailable pods during update",
                    default=0,
                    ge=0,
                    le=10,
                ),
            )
            .add(
                IntegerField(
                    "min_ready_seconds",
                    description="Minimum seconds for pod to be ready",
                    default=10,
                    ge=0,
                    le=300,
                ),
            ),
        ),
    ),
    Schema("observability", description="Observability and telemetry configuration")
    .add(
        StringField(
            "environment",
            description="Environment name for telemetry",
            default="production",
            enum=["development", "staging", "production"],
        ),
    )
    .add(
        AnyOf(
            Schema("metrics", description="Metrics collection")
            .add(
                BooleanField(
                    "enabled",
                    description="Enable metrics collection",
                    default=True,
                ),
            )
            .add(
                StringField(
                    "provider",
                    description="Metrics provider",
                    default="prometheus",
                    enum=["prometheus", "datadog", "cloudwatch", "newrelic"],
                ),
            )
            .add(
                IntegerField(
                    "scrape_interval_seconds",
                    description="Metrics scrape interval",
                    default=15,
                    ge=1,
                    le=300,
                ),
            )
            .add(
                Stringlist(
                    "custom_labels",
                    description="Custom metric labels",
                    default=["version", "region", "cluster"],
                    min_length=0,
                    max_length=50,
                ),
            ),
            Schema("logging", description="Centralized logging")
            .add(
                BooleanField(
                    "enabled",
                    description="Enable centralized logging",
                    default=True,
                ),
            )
            .add(
                StringField(
                    "backend",
                    description="Logging backend",
                    default="elasticsearch",
                    enum=["elasticsearch", "loki", "splunk", "cloudwatch"],
                ),
            )
            .add(
                StringField(
                    "log_level",
                    description="Minimum log level",
                    default="INFO",
                    enum=["DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"],
                ),
            )
            .add(
                IntegerField(
                    "retention_days",
                    description="Log retention period",
                    default=30,
                    ge=1,
                    le=365,
                ),
            ),
            Schema("tracing", description="Distributed tracing")
            .add(
                BooleanField(
                    "enabled",
                    description="Enable distributed tracing",
                    default=True,
                ),
            )
            .add(
                StringField(
                    "backend",
                    description="Tracing backend",
                    default="jaeger",
                    enum=["jaeger", "zipkin", "tempo", "xray"],
                ),
            )
            .add(
                FloatField(
                    "sampling_rate",
                    description="Trace sampling rate (0.0 to 1.0)",
                    default=0.1,
                    ge=0.0,
                    le=1.0,
                ),
            )
            .add(
                Stringlist(
                    "propagation_formats",
                    description="Trace context propagation formats",
                    default=["w3c", "b3"],
                    min_length=1,
                    max_length=5,
                ),
            ),
            Schema("profiling", description="Continuous profiling")
            .add(
                BooleanField(
                    "enabled",
                    description="Enable continuous profiling",
                    default=False,
                ),
            )
            .add(
                StringField(
                    "provider",
                    description="Profiling provider",
                    default="pyroscope",
                    enum=["pyroscope", "pprof", "datadog"],
                )
            )
            .add(
                IntegerField(
                    "sample_rate_hz",
                    description="Profiling sample rate",
                    default=100,
                    ge=10,
                    le=1000,
                ),
            ),
        ),
    ),
    Schema("security", description="Security and compliance configuration")
    .add(
        Schema("encryption", description="Encryption settings")
        .add(
            Byteslist(
                "encryption_keys",
                description="Encryption keys for data at rest (base64 encoded)",
                default=[
                    b"a2V5LTEtYmFzZTY0LWVuY29kZWQ=",
                    b"a2V5LTItYmFzZTY0LWVuY29kZWQ=",
                ],
                min_length=1,
                max_length=5,
            ),
        )
        .add(
            IntegerField(
                "key_rotation_days",
                description="Key rotation period",
                default=90,
                ge=1,
                le=365,
            ),
        )
        .add(
            BooleanField(
                "encrypt_at_rest",
                description="Enable encryption at rest",
                default=True,
            ),
        ),
    )
    .add(
        Schema("compliance", description="Compliance settings")
        .add(
            Stringlist(
                "frameworks",
                description="Compliance frameworks to enforce",
                default=["SOC2", "HIPAA", "GDPR"],
                min_length=1,
                max_length=10,
            ),
        )
        .add(
            DateField(
                "last_audit_date",
                description="Last compliance audit date",
                default=datetime(2025, 1, 15, tzinfo=timezone.utc),
            ),
        )
        .add(
            DateField(
                "next_audit_date",
                description="Next scheduled audit date",
                default=datetime(2025, 7, 15, tzinfo=timezone.utc),
            ),
        ),
    )
    .add(
        Schema("access_control", description="Access control policies")
        .add(
            BooleanField(
                "rbac_enabled",
                description="Enable role-based access control",
                default=True,
            ),
        )
        .add(
            Stringlist(
                "allowed_roles",
                description="Allowed IAM roles",
                default=["admin", "developer", "operator", "viewer"],
                min_length=1,
                max_length=50,
            ),
        )
        .add(
            Integerlist(
                "session_timeout_minutes",
                description="Session timeouts per role (maps to allowed_roles order)",
                default=[120, 60, 90, 30],
                min_length=1,
                max_length=50,
                item_ge=5,
                item_le=1440,
            ),
        ),
    ),
]

In [8]:
conifg_manager = Manager(*schemas)
conifg_manager.create_templates("templates/")
config = conifg_manager.load("config")

In [9]:
config.deployment

Config_Deployment(next_deployment_window=datetime.datetime(2025, 10, 15, 2, 0, tzinfo=datetime.timezone.utc), maintenance_windows=[datetime.datetime(2025, 10, 15, 2, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2025, 11, 15, 2, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2025, 12, 15, 2, 0, tzinfo=datetime.timezone.utc)], blue_green=Config_Deployment_Blue_green(verification_time_minutes=15, auto_rollback=True, error_threshold_percent=5.0))